In [1]:
import numpy as np
import tensorflow as tf

from keras.preprocessing.sequence import pad_sequences
import pickle

Using TensorFlow backend.


# Check if tf-gpu is enabled

If not : 
and
tg-gpu=2.1.0
keras : conda keras=2.3.1

In [2]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

tf.__version__

Default GPU Device:/device:GPU:0


'2.1.0'

In [3]:
import data_utils
import glove_utils
import models
import display_utils
from goog_lm import LM

In [4]:
import lm_data_utils
import lm_utils

In [5]:
np.random.seed(1002)
tf.compat.v1.set_random_seed
(1002)

1002

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
VOCAB_SIZE  = 50000
with open('aux_files/dataset_%d.pkl' %VOCAB_SIZE, 'rb') as f:
    dataset = pickle.load(f)

In [8]:
doc_len = [len(dataset.test_seqs2[i]) for i in 
           range(len(dataset.test_seqs2))]

In [9]:
dist_mat = np.load('aux_files/dist_counter_%d.npy' %VOCAB_SIZE)
# Prevent returning 0 as most similar word because it is not part of the dictionary
dist_mat[0,:] = 100000
dist_mat[:,0] = 100000

skip_list = np.load('aux_files/missed_embeddings_counter_%d.npy' %VOCAB_SIZE)

### Demonstrating how we find the most similar words

In [10]:
for i in range(300, 305):
    src_word = i
    nearest, nearest_dist = glove_utils.pick_most_similar_words(src_word, dist_mat,20, 0.5)
        
    print('Closest to `%s` are:' %(dataset.inv_dict[src_word]))
    for w_id, w_dist in zip(nearest, nearest_dist):
          print(' -- ', dataset.inv_dict[w_id], ' ', w_dist)

    print('----')

Closest to `later` are:
 --  subsequent   0.1832310977140006
 --  subsequently   0.18671959913400138
 --  afterward   0.2509214012219996
 --  afterwards   0.2576958961479985
 --  thereafter   0.27419810965900004
 --  trailing   0.336800271281001
 --  after   0.3452026123779999
 --  then   0.3647283933829992
 --  posterior   0.4310855888389993
 --  following   0.48330736760400095
----
Closest to `takes` are:
 --  pick   0.3113054656320011
 --  taking   0.42471158462800096
 --  picked   0.4852741249590009
----
Closest to `instead` are:
 --  conversely   0.3034038049850012
 --  however   0.3475382865829997
 --  alternatively   0.39540487543000036
 --  alternately   0.4439627395600003
 --  nevertheless   0.47716397579200054
----
Closest to `seem` are:
 --  seems   0.007052995653000105
 --  appears   0.32837244735200044
 --  looks   0.33534638306400066
 --  transpires   0.45620718549300143
----
Closest to `john` are:
 --  jon   0.266846971000001
 --  giovanni   0.28136114290399905
 --  joha

### Preparing the dataset

In [11]:
max_len = 250
train_x = pad_sequences(dataset.train_seqs2, maxlen=max_len, padding='post')
train_y = np.array(dataset.train_y)
test_x = pad_sequences(dataset.test_seqs2, maxlen=max_len, padding='post')
test_y = np.array(dataset.test_y)

### Loading the sentiment analysis model

In [12]:
tf.compat.v1.disable_eager_execution()
tf.compat.v1.reset_default_graph()
if tf.compat.v1.get_default_session():
    sess.close()
sess = tf.compat.v1.Session()
batch_size = 1
lstm_size = 128
#max_len =  100

with tf.compat.v1.variable_scope('imdb', reuse=False):
    model = models.SentimentModel(batch_size=batch_size,
                           lstm_size = lstm_size,
                           max_len = max_len,
                           embeddings_dim=300, vocab_size=dist_mat.shape[1],is_train = False)
saver = tf.compat.v1.train.Saver()
saver.restore(sess, './models/imdb_model_final')



Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from ./models/imdb_model_final


In [47]:
with open('attack_results_final_threshold2_attack_with_defense_2_.pkl', 'rb') as f:
    (test_list, orig_list, label_list, adv_list, dist_list) = pickle.load( f)

idx = 7
if adv_list[idx] is None:
    print('Failed')
else:
    
    display_utils.visualize_attack(sess, model, dataset, orig_list[visual_idx], adv_list[visual_idx])
    #display_utils.visualize_attack2(dataset, test_list[idx], orig_list[idx], adv_list[idx], label_list[idx])

TypeError: 'NoneType' object is not subscriptable

## Loading the Google Language model

In [13]:
goog_lm = LM()

goog_lm/vocab-2016-09-10.txt
LM vocab loading done
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:42:00.0, compute capability: 6.1



Recovering graph.


INFO:tensorflow:Recovering Graph goog_lm/graph-2016-09-10.pbtxt


Recovering checkpoint goog_lm/ckpt-*


#### demonstrating the GoogLM

In [14]:
src_word = dataset.dict['play']
nearest, nearest_dist = glove_utils.pick_most_similar_words(src_word, dist_mat,20)
nearest_w = [dataset.inv_dict[x] for x in nearest]
print('Closest to `%s` are %s' %(dataset.inv_dict[src_word], nearest_w))

Closest to `play` are ['playing', 'gaming', 'games', 'toy', 'playback', 'game', 'plaything', 'cheek', 'gambling', 'toys', 'toying', 'replay', 'stake', 'plays', 'jeu', 'gamble', 'staking', 'reproduction', 'casino', 'sets']


In [15]:
prefix = 'is'
suffix = 'with'
lm_preds = goog_lm.get_words_probs(prefix, nearest_w, suffix)
print('most probable is ', nearest_w[np.argmax(lm_preds)])


most probable is  game


## Try Attack

In [16]:
from attacks import GeneticAtack

## Main Attack 

In [32]:
pop_size = 60
n1 = 8

with tf.compat.v1.variable_scope('imdb', reuse=True):
    batch_model = models.SentimentModel(batch_size=pop_size,
                           lstm_size = lstm_size,
                           max_len = max_len,
                           embeddings_dim=300, vocab_size=dist_mat.shape[1],is_train = False)
    
with tf.compat.v1.variable_scope('imdb', reuse=True):
    neighbour_model = models.SentimentModel(batch_size=n1,
                           lstm_size = lstm_size,
                           max_len = max_len,
                           embeddings_dim=300, vocab_size=dist_mat.shape[1],is_train = False)
ga_atttack = GeneticAtack(sess, model, batch_model, neighbour_model, dataset, dist_mat, 
                                  skip_list,
                                  goog_lm, max_iters=10,#####20, ####30, 
                                   pop_size=pop_size,
                                  
                                  n1 = n1,
                                  n2 = 4,
                                 use_lm = True, use_suffix=False)

In [42]:
import past_queries

past_queries.index = 0
past_queries.number_attack_detected = 0 
past_queries.number_pred = 0
past_queries.queries = np.ones((100,300),dtype=np.uint8)*-1
past_queries.outputs = np.ones((100,2), dtype=np.float32)*-1


SAMPLE_SIZE = 1#10#5000
#TEST_SIZE = 200
TEST_SIZE= 3
np.random.seed(1002)
#test_idx = np.random.choice(len(dataset.test_y), SAMPLE_SIZE, replace=False)
#test_idx = np.random.choice(len(dataset.test_y), SAMPLE_SIZE, replace=False)
test_idx = [22286,1207, 3394, 12042, 9468, 11169, 2414, 4320, 22286, 14231, 2259]

test_len = []

for i in range(SAMPLE_SIZE):
    test_len.append(len(dataset.test_seqs2[test_idx[i]]))
print('Shortest sentence in our test set is %d words' %np.min(test_len))

test_list = []
orig_list = []
orig_label_list = []
adv_list = []
dist_list = []

for i in range(SAMPLE_SIZE):
    x_orig = test_x[test_idx[i]]
    orig_label = test_y[test_idx[i]]
    orig_preds=  model.predict(sess, x_orig[np.newaxis, :])[0]
    # print(orig_label, orig_preds, np.argmax(orig_preds))
    if np.argmax(orig_preds) != orig_label:
        #print('skipping wrong classifed ..')
        #print('--------------------------')
        continue
    x_len = np.sum(np.sign(x_orig))
    if x_len >= 100:
        #print('skipping too long input..')
        #print('--------------------------')
        continue
    # if np.max(orig_preds) < 0.90:
    #    print('skipping low confidence .. \n-----\n')
    #    continue
    print('****** ', len(test_list) + 1, ' ********')
    test_list.append(test_idx[i])
    orig_list.append(x_orig)
    target_label = 1 if orig_label == 0 else 0
    orig_label_list.append(orig_label)
    x_adv = ga_atttack.attack( x_orig, target_label)
    adv_list.append(x_adv)
    if x_adv is None:
        print("Attack - Pred", past_queries.number_attack_detected, past_queries.number_pred )
        past_queries.number_attack_detected = 0 
        past_queries.number_pred = 0
        print('%d failed' %(i+1))
        dist_list.append(100000)
    else:
        print("Attack - Pred", past_queries.number_attack_detected, past_queries.number_pred )
        past_queries.number_attack_detected = 0 
        past_queries.number_pred = 0
        num_changes = np.sum(x_orig != x_adv)
        print('%d - %d changed.' %(i+1, num_changes))
        dist_list.append(num_changes)
        # display_utils.visualize_attack(sess, model, dataset, x_orig, x_adv)
    print('--------------------------')
    if (len(test_list)>= TEST_SIZE):
        break

Shortest sentence in our test set is 50 words
******  1  ********
		 0  --  0.6655526
Attack - Pred 0 601
1 - 1 changed.
--------------------------


## Compute Attack success rate

In [43]:
orig_len = [np.sum(np.sign(x)) for x in orig_list]
normalized_dist_list = [dist_list[i]/orig_len[i] for i in range(len(orig_list)) ]

In [44]:
SUCCESS_THRESHOLD  = 0.25
successful_attacks = [x < SUCCESS_THRESHOLD for x in normalized_dist_list]
print('Attack success rate : {:.2f}%'.format(np.mean(successful_attacks)*100))
print('Median percentange of modifications: {:.02f}% '.format(
    np.median([x for x in normalized_dist_list if x < 1])*100))
print('Mean percentange of modifications: {:.02f}% '.format(
    np.mean([x for x in normalized_dist_list if x < 1])*100))

Attack success rate : 100.00%
Median percentange of modifications: 2.00% 
Mean percentange of modifications: 2.00% 


In [36]:
visual_idx = np.random.choice(len(orig_list))
display_utils.visualize_attack(sess, model, dataset, orig_list[visual_idx], adv_list[visual_idx])

TypeError: 'NoneType' object is not subscriptable

In [37]:
visual_idx = np.random.choice(len(orig_list))
display_utils.visualize_attack(sess, model, dataset, orig_list[visual_idx], adv_list[visual_idx])

TypeError: 'NoneType' object is not subscriptable

In [45]:
visual_idx = np.random.choice(len(orig_list))
display_utils.visualize_attack(sess, model, dataset, orig_list[visual_idx], adv_list[visual_idx])

[11, 17, 284, 69, 12, 1528, 25, 54, 1853, 41, 1, 901, 8, 1, 616, 20084, 595, 322, 588, 45, 22, 178, 5, 64, 42400, 99, 16, 11, 754, 103, 10668, 2, 22, 77, 64, 12, 1334, 6, 75, 2, 1334, 6, 49, 8, 11, 1537, 4, 1671, 2, 15912]
[11, 17, 284, 69, 12, 1528, 25, 54, 1853, 41, 1, 901, 8, 1, 616, 20084, 595, 322, 588, 45, 22, 178, 5, 64, 42400, 99, 16, 11, 754, 103, 10668, 2, 22, 77, 64, 12, 1334, 6, 3811, 2, 1334, 6, 49, 8, 11, 1537, 4, 1671, 2, 15912]
Original Prediction = Negative. (Confidence = 59.59) 


---------  After attack -------------
New Prediction = Positive. (Confidence = 66.56) 


In [38]:
test_list

[1207, 3394, 12042]

In [26]:
visual_idx

6

In [40]:
## Save success
with open('attack_results_ONE_THRESHOLD.pkl', 'wb') as f:
    pickle.dump((test_list, orig_list, orig_label_list, adv_list, normalized_dist_list), f)
    
